In [18]:
source('cascade_helper.R')


In [2]:
p_thr <- list(1e-7, 1e-7, 5e-8)
names(p_thr) <- c('protein-truncating', 'protein-altering', 'non-coding')
p_thr_df <- as.data.frame(p_thr) %>% gather(Csq, p_thr) %>% mutate(Csq = str_replace_all(Csq, '\\.', '-'))


In [3]:
outd <- file.path(getwd(), 'out')


In [4]:
annot.tbl <- '/oak/stanford/groups/mrivas/private_data/ukbb/variant_filtering/variant_filter_table.tsv.gz'
annot.arr <- read_annot_arr(annot.tbl)


In [5]:
annot.arr %>% count(Csq)


Csq,n
<chr>,<int>
non-coding,669033
protein-altering,87407
protein-truncating,27816


In [6]:
array.loci <- annot.arr %>% 
filter(Csq %in% c('protein-altering', 'protein-truncating')) %>% 
select(CHROM, POS, REF, ALT, Csq)


In [8]:
traits <- fread('cascade.input.files.tsv') %>% drop_na(c('imp')) %>% filter(imp != '')


In [19]:
imp_anno_df <- traits %>% 
read_imp_sumstats_all(p_thr[['non-coding']]) %>%
annotate_imp_df()


In [21]:
imp_anno_df %>% count(name)

name,n
<chr>,<int>
Alanine aminotransferase,197
Albumin,206
Alkaline phosphatase,291
Apolipoprotein A,216
Apolipoprotein B,223
Aspartate aminotransferase,237
C-reactive protein,185
Calcium,202
Cholesterol,236


In [22]:
imp_anno_df %>%
fwrite(file.path(outd, 'cascade.imp.hits.tsv'), sep='\t')


In [27]:
imp_anno_df %>% 
select(is_rare, is_outside_of_MHC, ID) %>%
unique() %>%
count(is_outside_of_MHC, is_rare)

is_outside_of_MHC,is_rare,n
<lgl>,<lgl>,<int>
FALSE,FALSE,57
TRUE,FALSE,5112
NA,FALSE,2


In [28]:
imp_anno_df %>% 
filter(is.na(is_outside_of_MHC))

ID,REF,ALT,Effect,StdErr,Direction,HetISq,HetChiSq,HetDf,HetPVal,⋯,CHROM,POS,P,CM,id,prev,block,name,is_outside_of_MHC,is_rare
<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<int>,<dbl>,⋯,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<chr>,<lgl>,<lgl>
X:31854782_G_C,G,C,0.0121,0.0021,++-+,44.5,5.408,3,0.1443,⋯,X,31854782,6.421e-09,43.57216,3,25.25451,2,Urea,NA,FALSE
X:31917454_C_CT,C,CT,0.0112,0.0020,+++?,0.0,0.034,2,0.9829,⋯,X,31917454,3.630e-08,43.64179,1,0.00000,1,Sodium in urine,NA,FALSE


```
gdrive upload -p 1fGlnrj8Zu1Pox3fmxmFKdcbNVWiN9vhd out/cascade.imp.hits.tsv
zstd -15 --rm out/cascade.imp.hits.tsv
```